## 버거킹 메뉴 추천 ChatGPT

&nbsp;

In [2]:
!pip install langchain
!pip install pypdf
!pip install openai
!pip install gdown
!pip install chromadb
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.7/960.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.9 MB/s eta 0:00:00


### 사용할 패키지 불러오기

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from glob import glob
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pypdf import PdfReader
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import RetrievalQAWithSourcesChain
import gdown
openai_api_key = "api-key"

&nbsp;

### PDF 불러오기 및 전처리

In [14]:
id = "11U7let6PY_YCJpgRT0Dpr5DXSO3Ceqep"
file_name = "버거킹.pdf"
gdown.download(id=id, output=file_name, quiet=False)

loader = PyPDFLoader(file_name)
documents = loader.load()

Downloading...
From: https://drive.google.com/uc?id=11U7let6PY_YCJpgRT0Dpr5DXSO3Ceqep
To: /content/버거킹.pdf
100%|██████████| 346k/346k [00:00<00:00, 112MB/s]


In [15]:
documents

[Document(page_content='서초동 로그인\n인기메뉴\n스페셜오퍼버거킹-강남대로점\n사장님알림  딜리버리 서비스 메뉴의 가격은 매장 가격과 상이할 수 있습니다.★★★★★ 4.7\n최소주문금액 12,000원\n결제 요기서결제\n메뉴 105 클린리뷰 1404 정보\n콰트로 맥시멈…\n20,000원콰트로 맥시멈…\n18,000원스모키 바비큐 …\n13,500원스모키 바비큐 …\n12,000원치킨킹BLT 세\n10,900원\n통새우와퍼세트\n입안에서 톡톡 터지는 통새우가 매콤한 스파이시 토마토소스와 100% 순쇠고기 패티를 만나 완성한\n통새우와퍼!\n11,300원\n콰트로치즈와퍼 세트\n네가지 고품격 치즈와 100% 순쇠고기 패티로 탄생한 버거킹의 스테디 셀러, 콰트로치즈와퍼!\n11,300원\n갈릭불고기와퍼 세트\n크리스피 갈릭칩과 달콤한 불고기소스로 즐기는 갈릭불고기와퍼!\n10,800원\n와퍼 세트\n불에 직접 구운 순 쇠고기 패티에 싱싱한 야채가 한가득 버거킹의 대표 메뉴, 와퍼!\n10,500원\n몬스터와퍼세트\n디아블로 소스의 매콤함, 불에 직접 구운 100% 순쇠고기 패티와 치킨패티까지 압도적 크기의 몬스\n터와퍼!\n12,200원\n기간한정\n바삭킹 & 너겟킹10,스위트칠리소스\n바삭킹8조각＋너겟킹10조각＋스위트칠리소스\n16,000원\n바삭킹 & 너겟킹10,디아블로소스\n바삭킹8조각＋너겟킹10조각＋디아블로소스\n16,000원\n바삭킹 & 모짜볼10,스위트칠리소스\n바삭킹8조각＋크리미모짜볼10조각＋스위트칠리소스\n15,900원\n주문표 바로 주문하기', metadata={'source': '버거킹.pdf', 'page': 0}),
 Document(page_content='와퍼 세트\nX 세트바삭킹 & 모짜볼10,디아블로소스\n바삭킹8조각＋크리미모짜볼10조각＋디아블로소스\n15,900원\n콰트로 맥시멈…이하생략 3 세트\n100% 순쇠고기 패티 3장에 네 가지 치즈까지 더한 압도적인 녀석이 나타났다.\n18,000원\n콰트로 맥시멈…이하

In [16]:
output = []
# text 정제
for page in documents:
    text = page.page_content
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)   # 안녕-\n하세요 -> 안녕하세요  
    text = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", text.strip()) # "인\n공\n\n지능펙\n토리 -> 인공지능펙토리
    text = re.sub(r"\n\s*\n", "\n\n", text) # \n버\n\n거\n\n킹\n -> 버\n거\n킹
    output.append(text)

In [17]:
output

['서초동 로그인 인기메뉴 스페셜오퍼버거킹-강남대로점 사장님알림  딜리버리 서비스 메뉴의 가격은 매장 가격과 상이할 수 있습니다.★★★★★ 4.7 최소주문금액 12,000원 결제 요기서결제 메뉴 105 클린리뷰 1404 정보 콰트로 맥시멈… 20,000원콰트로 맥시멈… 18,000원스모키 바비큐 … 13,500원스모키 바비큐 … 12,000원치킨킹BLT 세 10,900원 통새우와퍼세트 입안에서 톡톡 터지는 통새우가 매콤한 스파이시 토마토소스와 100% 순쇠고기 패티를 만나 완성한 통새우와퍼! 11,300원 콰트로치즈와퍼 세트 네가지 고품격 치즈와 100% 순쇠고기 패티로 탄생한 버거킹의 스테디 셀러, 콰트로치즈와퍼! 11,300원 갈릭불고기와퍼 세트 크리스피 갈릭칩과 달콤한 불고기소스로 즐기는 갈릭불고기와퍼! 10,800원 와퍼 세트 불에 직접 구운 순 쇠고기 패티에 싱싱한 야채가 한가득 버거킹의 대표 메뉴, 와퍼! 10,500원 몬스터와퍼세트 디아블로 소스의 매콤함, 불에 직접 구운 100% 순쇠고기 패티와 치킨패티까지 압도적 크기의 몬스 터와퍼! 12,200원 기간한정 바삭킹 & 너겟킹10,스위트칠리소스 바삭킹8조각＋너겟킹10조각＋스위트칠리소스 16,000원 바삭킹 & 너겟킹10,디아블로소스 바삭킹8조각＋너겟킹10조각＋디아블로소스 16,000원 바삭킹 & 모짜볼10,스위트칠리소스 바삭킹8조각＋크리미모짜볼10조각＋스위트칠리소스 15,900원 주문표 바로 주문하기',
 '와퍼 세트 X 세트바삭킹 & 모짜볼10,디아블로소스 바삭킹8조각＋크리미모짜볼10조각＋디아블로소스 15,900원 콰트로 맥시멈…이하생략 3 세트 100% 순쇠고기 패티 3장에 네 가지 치즈까지 더한 압도적인 녀석이 나타났다. 18,000원 콰트로 맥시멈…이하생략 4 세트 100% 순쇠고기 패티 4장에 네 가지 치즈까지 더한 압도적인 녀석이 나타났다. 20,000원 스모키 바비큐 몬스터 세트 진한 훈연향과 불맛의 끝판왕 스모키바비큐, 바삭하게 튀긴 치킨패티까지 더해진 강력한 조합 13,500원 스

In [18]:
doc_chunks = []

for line in output:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, # 최대 청크 길이
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""], #  텍스트를 청크로 분할하는 데 사용되는 문자 목록
        chunk_overlap=0, # 인접한 청크 간에 중복되는 문자 수
    )
    chunks = text_splitter.split_text(line)
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk, metadata={"page": i, "source": file_name}
        )
        doc_chunks.append(doc)

In [19]:
doc_chunks

[Document(page_content='서초동 로그인 인기메뉴 스페셜오퍼버거킹-강남대로점 사장님알림  딜리버리 서비스 메뉴의 가격은 매장 가격과 상이할 수 있습니다.★★★★★ 4.7 최소주문금액 12,000원 결제 요기서결제 메뉴 105 클린리뷰 1404 정보 콰트로 맥시멈… 20,000원콰트로 맥시멈… 18,000원스모키 바비큐 … 13,500원스모키 바비큐 … 12,000원치킨킹BLT 세 10,900원 통새우와퍼세트 입안에서 톡톡 터지는 통새우가 매콤한 스파이시 토마토소스와 100% 순쇠고기 패티를 만나 완성한 통새우와퍼! 11,300원 콰트로치즈와퍼 세트 네가지 고품격 치즈와 100% 순쇠고기 패티로 탄생한 버거킹의 스테디 셀러, 콰트로치즈와퍼! 11,300원 갈릭불고기와퍼 세트 크리스피 갈릭칩과 달콤한 불고기소스로 즐기는 갈릭불고기와퍼! 10,800원 와퍼 세트 불에 직접 구운 순 쇠고기 패티에 싱싱한 야채가 한가득 버거킹의 대표 메뉴, 와퍼! 10,500원 몬스터와퍼세트 디아블로 소스의 매콤함, 불에 직접 구운 100% 순쇠고기 패티와 치킨패티까지 압도적 크기의 몬스 터와퍼! 12,200원 기간한정 바삭킹 & 너겟킹10,스위트칠리소스 바삭킹8조각＋너겟킹10조각＋스위트칠리소스 16,000원 바삭킹 & 너겟킹10,디아블로소스 바삭킹8조각＋너겟킹10조각＋디아블로소스 16,000원 바삭킹 & 모짜볼10,스위트칠리소스 바삭킹8조각＋크리미모짜볼10조각＋스위트칠리소스 15,900원 주문표 바로 주문하기', metadata={'page': 0, 'source': '버거킹.pdf'}),
 Document(page_content='와퍼 세트 X 세트바삭킹 & 모짜볼10,디아블로소스 바삭킹8조각＋크리미모짜볼10조각＋디아블로소스 15,900원 콰트로 맥시멈…이하생략 3 세트 100% 순쇠고기 패티 3장에 네 가지 치즈까지 더한 압도적인 녀석이 나타났다. 18,000원 콰트로 맥시멈…이하생략 4 세트 100% 순쇠고기 패티 4장에 네 가지 치즈까지 더한 압도적인 녀석이 

In [26]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
index = Chroma.from_documents(doc_chunks, embeddings)

&nbsp;

### Prompt 및 chain 설정

In [33]:
system_template="""To answer the question at the end, use the following context. If you don't know the answer, just say you don't know and don't try to make up an answer.
I want you to act as my Burger King menu recommender. It tells you your budget and suggests what to buy. You should only reply to items you recommend. Don't write a description.

Below is an example.
“My budget is 10,000 won, and it is the best menu combination within the budget."

you only answer in Korean

{summaries}
"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)


chain_type_kwargs = {"prompt": prompt}
bk_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0,openai_api_key=openai_api_key), 
    chain_type="stuff", 
    retriever=index.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
    reduce_k_below_max_tokens=True
)

&nbsp;

### 답변 테스트

In [28]:
result = bk_chain({"question": '총 예산 30,000원을 넘지않게 메뉴조합, 조합가격, 메뉴설명을 수행해주세요.'})

print(f"질문 : {result['question']}")
print()
print(f"답변 : {result['answer']}")

질문 : 총 예산 30,000원을 넘지않게 메뉴조합, 조합가격, 메뉴설명을 수행해주세요.

답변 : 30,000원 이내로 추천해드리겠습니다.

1. 와퍼 세트 (10,500원) + 너겟킹 8조각 (5,300원) + 쉐이킹프라이 (3,300원) = 19,100원
- 와퍼 세트: 불에 직접 구운 순 쇠고기 패티에 싱싱한 야채가 한가득 버거킹의 대표 메뉴, 와퍼!
- 너겟킹 8조각: 바삭하고 촉촉한 부드러운 너겟킹!
- 쉐이킹프라이: 구운갈릭 깊은 풍미가 느껴지는 시즈닝으로 취향 저격! 구운 마늘의 깊은 풍미!

2. 콰트로치즈와퍼 세트 (11,300원) + 어니언링 (3,300원) + 너겟킹 4조각 (3,100원) = 17,700원
- 콰트로치즈와퍼 세트: 네가지 고품격 치즈와 100% 순쇠고기 패티로 탄생한 버거킹의 스테디 셀러, 콰트로치즈와퍼!
- 어니언링: 크기도 UP! 맛도 UP! 고소한 진한 풍미가 일품인 어니언링!
- 너겟킹 4조각: 바삭하고 촉촉한 부드러운 너겟킹!

3. 통새우와퍼 세트 (11,300원) + 21치즈스틱 (3,400원) + 스윗어니언 시즈닝 (300원) = 15,000원
- 통새우와퍼 세트: 입안에서 톡톡 터지는 통새우가 매콤한 스파이시 토마토소스와 100% 순쇠고기 패티를 만나 완성한 통새우와퍼!
- 21치즈스틱: 21cm의 역대급 사이즈! 진하고 고소한 자연 모짜렐라가 가득한 21치즈스틱!
- 스윗어니언 시즈닝: 고소한 치즈맛에 매콤함까지!!

위의 조합 중에서 원하시는 메뉴를 선택하시면 됩니다.


In [29]:
result = bk_chain({"question": '총 예산 20,000원을 넘지않게 메뉴조합, 조합가격, 메뉴설명을 수행해주세요.'})

print(f"질문 : {result['question']}")
print()
print(f"답변 : {result['answer']}")

질문 : 총 예산 20,000원을 넘지않게 메뉴조합, 조합가격, 메뉴설명을 수행해주세요.

답변 : 콰트로 맥시멈 세트와 너겟킹 8조각을 조합하면 총 19,700원으로 예산 내에 들어갑니다. 콰트로 맥시멈 세트는 네 가지 고품격 치즈와 100% 순쇠고기 패티로 탄생한 버거킹의 스테디 셀러이며, 너겟킹 8조각은 바삭하고 촉촉한 부드러운 너겟킹입니다.


In [30]:
result = bk_chain({"question": '총 예산 10,000원을 넘지않게 메뉴조합, 조합가격, 메뉴설명을 수행해주세요.'})

print(f"질문 : {result['question']}")
print()
print(f"답변 : {result['answer']}")

질문 : 총 예산 10,000원을 넘지않게 메뉴조합, 조합가격, 메뉴설명을 수행해주세요.

답변 : 제가 추천하는 메뉴 조합은 "와퍼주니어"와 "너겟킹4조각"입니다. 이 조합의 가격은 9,500원입니다. 와퍼주니어는 불에 직접 구운 순 쇠고기 패티에 싱싱한 야채가 한가득 들어간 대표 메뉴이며, 너겟킹은 바삭하고 촉촉한 부드러운 너겟킹입니다. 두 메뉴를 함께 즐기면서 다양한 맛을 느낄 수 있습니다.


In [31]:
result = bk_chain({"question": '사이드메뉴 추천해줘'})

print(f"질문 : {result['question']}")
print()
print(f"답변 : {result['answer']}")

질문 : 사이드메뉴 추천해줘

답변 : 메뉴에는 다양한 사이드메뉴가 있지만, 제가 추천하는 사이드메뉴는 쉐이킹프라이입니다. 구운 마늘의 깊은 풍미와 시즈닝으로 취향을 저격할 수 있는 맛이 특징입니다. 가격도 3,300원으로 저렴하고, 버거와 함께 즐기기에도 좋습니다.


&nbsp;

### chat-gpt 사용시 주의 사항
- https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa_with_sources.html

In [35]:
help(RetrievalQAWithSourcesChain)

Help on class RetrievalQAWithSourcesChain in module langchain.chains.qa_with_sources.retrieval:

class RetrievalQAWithSourcesChain(langchain.chains.qa_with_sources.base.BaseQAWithSourcesChain)
 |  RetrievalQAWithSourcesChain(*, memory: Optional[langchain.schema.BaseMemory] = None, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None, verbose: bool = None, combine_documents_chain: langchain.chains.combine_documents.base.BaseCombineDocumentsChain, question_key: str = 'question', input_docs_key: str = 'docs', answer_key: str = 'answer', sources_answer_key: str = 'sources', return_source_documents: bool = False, retriever: langchain.schema.BaseRetriever, reduce_k_below_max_tokens: bool = False, max_tokens_limit: int = 3375) -> None
 |  
 |  Question-answering with sources over an index.
 |  
 |  Method resolution order:
 |      Retr

In [32]:
result = bk_chain({"question": '와퍼주니어 설명해줘'})


print(f"질문 : {result['question']}")
print('----------------------------')
print(f"답변 : {result['answer']}")

질문 : 와퍼주니어 설명해줘
----------------------------
답변 : 와퍼주니어는 버거킹의 대표적인 메뉴 중 하나로, 불에 직접 구운 100% 순 쇠고기 패티에 싱싱한 야


In [34]:
result = bk_chain({"question": '와퍼주니어 설명해줘'})


print(f"질문 : {result['question']}")
print('----------------------------')
print(f"답변 : {result['answer']}")

질문 : 와퍼주니어 설명해줘
----------------------------
답변 : 와퍼주니어는 버거킹의 대표적인 메뉴 중 하나로, 작지만 꽉 찬 100% 순 쇠고기 패티가 들어간 버거입니다. 불에 직접 구워져서 고소한 맛이 나며, 싱싱한 야채와 함께 새콤한 마요네즈 소스가 들어가 맛을 더해줍니다. 와퍼보다는 작지만, 가격도 저렴하고 먹기에 편리한 사이즈로 인기가 많습니다.


&nbsp;

### 정리

1. Prompt의 범위를 너무 넓게 잡지않기
2. Prompt의 재현율이 낮으면 act, example로 원하는 형태의 결과를 재현할수 있도록 Prompt 추가
3. 사용하는 chain에 대한 이해 (help() 나 langchain 공식문서)
4. 잘 만들어진 다양한 Prompt 보기

https://github.com/f/awesome-chatgpt-prompts